### Sample notebook for Client Side Processing.

Create results folder if not present. This will be the path of our local collections.

In [1]:
import os
if not os.path.exists('./results'):
    os.mkdir('results')

Connect to openEO Platform to download some sample data

In [2]:
import openeo
conn = openeo.connect("openeo.cloud").authenticate_oidc('egi')

Authenticated using refresh token.


netCDF samples:

In [4]:
temporal_extent = ["2017-06-01","2017-07-01"]
spatial_extent = {
          "west": 11.08,
          "east": 11.16,
          "south": 46.05,
          "north": 46.07
        }
S2_collection = 'SENTINEL2_L2A'
bands = ['B02','B03','B04']
S2_data = conn.load_collection(S2_collection,
                               temporal_extent=temporal_extent,
                               bands=bands,
                               spatial_extent=spatial_extent)
S2_data.download('results/S2_L2A_data.nc')

In [5]:
temporal_extent = ["2017-06-01","2017-07-01"]
spatial_extent = {
          "west": 11.08,
          "east": 11.16,
          "south": 46.05,
          "north": 46.07
        }
S1_collection = 'SENTINEL1_GRD'
bands = ['VV','VH']
S1_data = conn.load_collection(S1_collection,
                               temporal_extent=temporal_extent,
                               bands=bands,
                               spatial_extent=spatial_extent)
S1_data.download('results/S1_GRD_data.nc')

geoTIFF samples:

note: I'm stuck with geoTIFFs, since I'm not able to correctly parse the band names yet, see https://github.com/corteva/rioxarray/issues/81#issuecomment-1253397197

So, only netCDFs now are listed as local collections.

In [15]:
temporal_extent = ["2017-06-01","2017-07-01"]
spatial_extent = {
          "west": 11.08,
          "east": 11.16,
          "south": 46.05,
          "north": 46.07
        }
S2_collection = 'SENTINEL2_L2A'
bands = ['B02','B03','B04']
S2_data = conn.load_collection(S2_collection,
                               temporal_extent=temporal_extent,
                               bands=bands,
                               spatial_extent=spatial_extent)
S2_data.max_time().download('results/S2_L2A_data.tiff')

Start with the local collections discovery

In [3]:
conn = openeo.connect("./results/")

./results/


In [4]:
conn.list_collections()

[{'stac_version': '1.0.0-rc.2',
  'type': 'Collection',
  'id': './results/S2_L2A_data.nc',
  'title': './results/S2_L2A_data.nc',
  'description': '',
  'license': '',
  'providers': [{'name': 'openEO platform', 'roles': ['producer'], 'url': ''}],
  'links': '',
  'extent': {'spatial': {'bbox': [[11.079252577441025,
      46.05000342740738,
      11.160823305139893,
      46.070000870424366]]},
   'temporal': {'interval': [['2017-06-03T00:00:00.000000000',
      '2017-07-01T00:00:00.000000000']]}},
  'cube:dimensions': {'t': {'type': 'temporal',
    'extent': ['2017-06-03T00:00:00.000000000',
     '2017-07-01T00:00:00.000000000']},
   'x': {'type': 'spatial',
    'axis': 'x',
    'extent': [660855.0, 667105.0],
    'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], PARAMETER["central_meridian", 9.0], PARAMETER["latitude_of_origin", 0.0], PARAMETER["scale_factor", 0.9996], PARAMETER["false_easting", 500000.0], PARAMETER["false_northing", 0.0], UNIT["m", 1.0], AXIS["Easting", EAST], AXIS["Northing", NORTH], AUTHORITY["EPSG","32632"]]'},
   'y': {'type': 'spatial',
    'axis': 'y',
    'extent': [5101705.0, 5104095.0],
    'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], PARAMETER["central_meridian", 9.0], PARAMETER["latitude_of_origin", 0.0], PARAMETER["scale_factor", 0.9996], PARAMETER["false_easting", 500000.0], PARAMETER["false_northing", 0.0], UNIT["m", 1.0], AXIS["Easting", EAST], AXIS["Northing", NORTH], AUTHORITY["EPSG","32632"]]'},
   'bands': {'type': 'bands', 'values': ['B02', 'B03', 'B04']}}},
 {'stac_version': '1.0.0-rc.2',
  'type': 'Collection',
  'id': './results/S1_GRD_data.nc',
  'title': './results/S1_GRD_data.nc',
  'description': '',
  'license': '',
  'providers': [{'name': 'openEO platform', 'roles': ['producer'], 'url': ''}],
  'links': '',
  'extent': {'spatial': {'bbox': [[11.079252577441025,
      46.05000342740738,
      11.160823305139893,
      46.070000870424366]]},
   'temporal': {'interval': [['2017-06-03T00:00:00.000000000',
      '2017-06-29T00:00:00.000000000']]}},
  'cube:dimensions': {'t': {'type': 'temporal',
    'extent': ['2017-06-03T00:00:00.000000000',
     '2017-06-29T00:00:00.000000000']},
   'x': {'type': 'spatial',
    'axis': 'x',
    'extent': [660855.0, 667105.0],
    'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], PARAMETER["central_meridian", 9.0], PARAMETER["latitude_of_origin", 0.0], PARAMETER["scale_factor", 0.9996], PARAMETER["false_easting", 500000.0], PARAMETER["false_northing", 0.0], UNIT["m", 1.0], AXIS["Easting", EAST], AXIS["Northing", NORTH], AUTHORITY["EPSG","32632"]]'},
   'y': {'type': 'spatial',
    'axis': 'y',
    'extent': [5101705.0, 5104095.0],
    'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 63781

In [5]:
conn.describe_collection('./results/S1_GRD_data.nc')

{'stac_version': '1.0.0-rc.2',
 'type': 'Collection',
 'id': './results/S1_GRD_data.nc',
 'title': './results/S1_GRD_data.nc',
 'description': '',
 'license': '',
 'providers': [{'name': 'openEO platform', 'roles': ['producer'], 'url': ''}],
 'links': '',
 'extent': {'spatial': {'bbox': [[11.079252577441025,
     46.05000342740738,
     11.160823305139893,
     46.070000870424366]]},
  'temporal': {'interval': [['2017-06-03T00:00:00.000000000',
     '2017-06-29T00:00:00.000000000']]}},
 'cube:dimensions': {'t': {'type': 'temporal',
   'extent': ['2017-06-03T00:00:00.000000000',
    '2017-06-29T00:00:00.000000000']},
  'x': {'type': 'spatial',
   'axis': 'x',
   'extent': [660855.0, 667105.0],
   'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], PARAMETER["central_meridian", 9.0], PARAMETER["latitude_of_origin", 0.0], PARAMETER["scale_factor", 0.9996], PARAMETER["false_easting", 500000.0], PARAMETER["false_northing", 0.0], UNIT["m", 1.0], AXIS["Easting", EAST], AXIS["Northing", NORTH], AUTHORITY["EPSG","32632"]]'},
  'y': {'type': 'spatial',
   'axis': 'y',
   'extent': [5101705.0, 5104095.0],
   'reference_system': 'PROJCS["WGS 84 / UTM zone 32N", GEOGCS["WGS 84", DATUM["World Geodetic System 1984", SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], PARAMETER["central_meridian", 9.0], PARAMETER["latitude_of_origin", 0.0], PARAMETER["scale_factor", 0.9996], PARAMETER["false_easting", 500000.0], PARAMETER["false_northing", 0.0], UNIT["m", 1.0], AXIS["Easting", EAST], AXIS["Northing", NORTH], AUTHORITY["EPSG","32632"]]'},
  'bands': {'type': 'bands', 'values': ['VV', 'VH']}}}

In [6]:
S1_datacube = conn.load_collection('./results/S1_GRD_data.nc')

In [7]:
S1_datacube = S1_datacube.reduce_dimension(dimension='t',reducer='mean')

With the process graph we need to find a way to call the python processes now...

In [8]:
S1_datacube.print_json()

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "id": "./results/S1_GRD_data.nc",
        "spatial_extent": null,
        "temporal_extent": null
      }
    },
    "reducedimension1": {
      "process_id": "reduce_dimension",
      "arguments": {
        "data": {
          "from_node": "loadcollection1"
        },
        "dimension": "t",
        "reducer": {
          "process_graph": {
            "mean1": {
              "process_id": "mean",
              "arguments": {
                "data": {
                  "from_parameter": "data"
                }
              },
              "result": true
            }
          }
        }
      },
      "result": true
    }
  }
}


In [9]:
S1_datacube.metadata